# Chargement des librairies

In [1]:
library(sparklyr)
library(tidyverse, warn.conflicts = FALSE)


Attaching package: ‘sparklyr’


The following object is masked from ‘package:SparkR’:

    collect


── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.0     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()      masks SparkR::arrange()
✖ dplyr::between()      masks SparkR::between()
✖ dplyr::coalesce()     masks SparkR::coalesce()
✖ dplyr::collect()      masks sparklyr::collect(), SparkR::collect()
✖ dplyr::contains()     masks tidyr::contains(), SparkR::contains()
✖ dplyr::count()        masks SparkR::count()
✖ dplyr::cume_dist()    masks SparkR::cume_dist()
✖ dplyr::d

# Connexion à Spark

In [2]:
config <- spark_config()
sc <- spark_connect(master = "yarn-cluster", config = config, version = '2.4.3')

# Chargement de la table des séjours

In [3]:
tbl_change_db(sc, 'cse_200007_work')

spark_read_parquet(sc, 'sejours_rea_48_heures', 'sejours_rea_48_heures')

# Source: spark<sejours_rea_48_heures> [?? x 137]
   encounter_num location_cd chads_vasc patient_num start_date_stay    
   <chr>         <chr>            <dbl> <chr>       <dttm>             
 1 175266599629… UFR:026514           5 -447761911… 2020-03-26 17:34:00
 2 -12168133162… UFR:047242           4 5030412272… 2020-04-28 00:22:00
 3 523133734746… UFR:021034           1 6307540937… 2020-08-20 15:02:00
 4 448845052564… UFR:010030           1 -762983600… 2015-10-25 21:25:00
 5 752809267244… UFR:014025           4 3096753449… 2016-03-26 10:48:00
 6 -52459538182… UFR:087260           2 4241449024… 2016-03-31 10:44:00
 7 514156883888… UFR:066228           1 1527229162… 2020-10-28 10:00:00
 8 -20511423772… UFR:095013           1 7998281594… 2020-11-28 13:10:00
 9 225592971885… UFR:014025           1 -700016733… 2013-09-26 14:31:00
10 870541130703… UFR:014025           5 7778804384… 2016-07-16 16:24:00
# … with more rows, and 132 more variables: end_date_stay <dttm>,
#   instance_num <ch

In [4]:
# Création des tibbles correspondant aux tables créées dans data_management

sejours_rea_48_heures <- sc %>% tbl('sejours_rea_48_heures')

# Sélection du schéma FA

In [5]:
tbl_change_db(sc, 'cse_200007_20210519')

# Définition des variables

## UFR de réanimation

In [6]:
liste_reas_med <- c('UFR:028091',  'UFR:014025', 'UFR:095013', 'UFR:011440', 'UFR:010030',
                    'UFR:010502', 'UFR:021034', 'UFR:0101', 'UFR:026151', 'UFR:053011', 'UFR:047057',
                    'UFR:066067', 'UFR:066228', 'UFR:068033', 'UFR:073290', 'UFR:076023', 'UFR:087260')

liste_reas_chir <- c('UFR:095073', 'UFR:095062', 'UFR:010025', 'UFR:010260', 'UFR:005269', 'UFR:005389', 'UFR:021262', 'UFR:0613', 
                     'UFR:026513', 'UFR:026514', 'UFR:047242', 'UFR:061726', 'UFR:066020', 'UFR:066120', 'UFR:066420', 'UFR:066520', 
                     'UFR:066620', 'UFR:073260', 'UFR:073261')

liste_reas_enlevees <- c('UFR:028402', 'UFR:010302', 'UFR:053036', 'UFR:026511', 'UFR:066320', 'UFR:0107')

liste_reas <- c(liste_reas_med, liste_reas_chir)

liste_reas_non_triees <- c(liste_reas, liste_reas_enlevees)

## Liste des UFR laissées de côté après réanalyse de la liste

In [7]:
### Soins continus : c('UFR:028402', 'UFR:010302', 'UFR:053036')
### Chirurgie cardiaque : c('UFR:026511', 'UFR:066320')

## Examens de laboratoire

### Codes des examens de laboratoire d'hémostase

In [8]:
#### AntiXa-HNF : LAB:A0350, LAB:G1986, LAB:E3109, 
#### AntiXa-HBPM : LAB:A0018, LAB:E3108
#### Danaparoïde sodique : LAB:A0019, LAB:A2662, LAB:H6291, LAB:D9103
#### TCA : LAB:A0265 (Patient), LAB:A0266 (Témoin), LAB:A1792 (Ratio)
#### INR : LAB:A0269, LAB:F5455
#### AOD : LAB:E5350 (Rivaroxaban), LAB:F8012 (Apixaban), LAB:H3248 (Edoxaban)

lab_antiXa_hnf <- c ('LAB:A0350', 'LAB:G1986', 'LAB:E3109')
lab_antiXa_hbpm <- c ('LAB:A0018', 'LAB:E3108')
lab_tca_ratio <- c ('LAB:A1792')
lab_aod <- c('LAB:E5350', 'LAB:F8012', 'LAB:H3248')

lab_tous_tests <- c(lab_antiXa_hnf,lab_antiXa_hbpm,lab_tca_ratio,lab_aod)
lab_tous_tests_sans_tca <- c(lab_antiXa_hnf,lab_antiXa_hbpm,lab_aod)

### Codes des autres examens de laboratoire pertinents

In [9]:
## Codes de l'Hb - NB B1946 et B1945 sont l'Hb des GdS veineux et artériels respectivement, donc 
## discuter la variabilité des valeurs entre GdS et NFS.
lab_hb <- c('LAB:A0163', 'LAB:C1549', 'LAB:A9882', 'LAB:C8745', 'LAB:B1946', 'LAB:B1945')

## Correspondance LOINC : 22664-7
lab_uree <- c('LAB:A0286', 'LAB:G3350')

# Correspondance LOINC : 3255-7
lab_fib <- c('LAB:A0126', 'LAB:A1114', 'LAB:A2667')

## Correspondance LOINC : 777-3
## Correspondance LOINC : 778-1
lab_plq <- c('LAB:A0230', 'LAB:A2538', 'LAB:B9896', 'LAB:G5746', 'LAB:G7727', 'LAB:G7728', 'LAB:G7833', 'LAB:A1598', 'LAB:A2539')

## Correspondance LOINC : 48066-5
lab_ddim <- c('LAB:B4199', 'LAB:C0474', 'LAB:C7882', 'LAB:F5402')

## Correspondance LOINC : 14682-9
lab_creat <- c('LAB:F9409','LAB:C0697','LAB:F9410','LAB:F2621','LAB:G7834','LAB:A7813','LAB:H4038','LAB:A0094')

lab_tous_autres <- c(lab_hb, lab_uree, lab_fib, lab_plq, lab_ddim, lab_creat)

## Variables des actes CCAM

In [10]:
ccam_iot <- 'CCAM:GELD004'
ccam_vm <- c('CCAM:GLLD015', 'CCAM:GLLD004', 'CCAM:GLLD008')
ccam_vasopresseur <- c('CCAM:EQLF003', 'CCAM:EQLF001')
ccam_eer <- c('CCAM:JVJF002', 'CCAM:JVJF005', 'CCAM:JVJB002')
ccam_ecmo <- 'CCAM:EQQP004'
ccam_transfusion <- c('CCAM:FELF011', 'CCAM:FELF004')
ccam_coagulopathie <- 'CCAM:FELF003'

## Variables des codages diagnostics CIM10

### Codes ischémiques

In [11]:
#### I63 : AIC
#### I74 : Embolie ou thrombose artérielle
#### I513 : Thrombose intra-cardiaque
#### K55 : Troubles vasculaires aigus de l'intestin
#### N28 : Infarctus rénal
#### D735 : Infarctus splénique

ischemie <- tibble(code = c('CIM10:I63', 'CIM10:I74', 'CIM10:I513', 'CIM10:K550', 'CIM10:N280', 'CIM10:D735'),
                   type = c('cerebrale', 'vasculaire', 'cardiaque', 'intestinal', 'renal', 'splenique'),
                   binaire_ischemie = c(1,1,1,1,1,1))

diag_biais_anticoag <- tibble(code = c('CIM10:I74', 'CIM10:I513', 'CIM10:K550', 'CIM10:I26', 'CIM10:Z952', 'CIM10:I80', 'CIM10:I81', 'CIM10:I82', 'CIM10:D685', 'CIM10:D686'),
                        diag = c('b_thrombose_arterielle', 'b_thrombose_cardiaque', 'b_ischemie_mesenterique', 'b_embolie_pulmonaire', 'b_valve_cardiaque', 
                                 'b_thrombose_veineuse', 'b_thrombose_veineuse', 'b_thrombose_veineuse', 'b_thrombophilie', 'b_thrombophilie'),
                        binaire_biais_anticoag = c(1,1,1,1,1,1,1,1,1,1))

In [12]:
#### I61 : Hémorragie intracérébrale
#### I62 : Autre hémorragies intracrâniennes
#### R04 : Hémorragie des voies respiratoires
#### T7952 : 
#### T810 : Hémorragie compliquant un acte à visée diagnostique ou thérapeutique
#### M250 : Hémarthrose
#### R58 : Hémorragie, non classée ailleurs
#### N328 : Autres affections précisées de la vessie (un peu large, ça ?)
#### R31 : Hématurie
#### K922 : Hémorragie gastro-intestinale
#### K920 : Hématémèse
#### K661 : Hémopéritoine
#### K250 : Ulcère aigu de l'estomac
#### K260 : Ulcère aigu du duodénum
#### K270 : Ulcère digestif de siège non précisé avec hémorragie
#### K280 : Ulcère gastro-jéjunal aigu avec hémorragie
#### K2X4 : Ulcère [de localisation correspondant au X] chronique ou non précisé, avec hémorragie

hemorragie <- tibble(code = c('CIM10:I61', 'CIM10:I62', 'CIM10:R04', 'CIM10:T7952', 'CIM10:T810',
                      'CIM10:M250', 'CIM10:R58', 'CIM10:N328', 'CIM10:R31', 'CIM10:K922',
                      'CIM10:K920', 'CIM10:K661', 'CIM10:K260', 'CIM10:K250', 'CIM10:K280',
                      'CIM10:K270', 'CIM10:K264', 'CIM10:K254', 'CIM10:K284', 'CIM10:K274'),
                     type = c('cerebrale', 'cerebrale', 'respiratoire', 'autre', 'iatrogene',
                              'articulaire', 'autre', 'urinaire', 'urinaire', 'digestif', 'digestif',
                              'digestif', 'digestif', 'digestif', 'digestif', 'digestif', 'digestif',
                              'digestif', 'digestif', 'digestif'),
                     binaire_hemorragie = c(1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1))

## Tables

### Tables de la FA

#### FA de novo

In [13]:
fa_de_novo <- sc %>% tbl('i2b2_observation_cim10') %>%
    select(location_cd, concept_cd, encounter_num, patient_num, start_date) %>%
    mutate(concept_cd_abr = str_sub(concept_cd, 1, 9)) %>%
    filter(concept_cd_abr == "CIM10:I48") %>%
    collect() %>%
    group_by(patient_num) %>%
    slice_min(start_date) %>%
    mutate(fadenovo = ifelse(location_cd %in% liste_reas, 1, 0)) %>%
    select(-c(concept_cd, concept_cd_abr, start_date)) %>%
    filter(location_cd %in% liste_reas) %>%
    distinct()

In [14]:
fa_de_novo <- copy_to(sc, fa_de_novo, overwrite = TRUE)

#### Codages de FA

In [15]:
codages_fa <- sc %>% tbl('i2b2_observation_cim10') %>%
    select(location_cd, concept_cd, encounter_num, patient_num, start_date) %>%
    mutate(concept_cd_abr = str_sub(concept_cd, 1, 9)) %>%
    filter(concept_cd_abr == "CIM10:I48") %>%
    select(-c(concept_cd, start_date, concept_cd_abr))

## Flowchart
### Séjours de réanimation à partir de la liste fournie par l'EDS

In [16]:
sc %>% tbl('i2b2_observation_ufr') %>% 
    filter(location_cd %in% liste_reas_non_triees) %>%
    count() %>% as_tibble()

n
<dbl>
17695


### Séjours de réanimation depuis une liste triée
Pas de chirurgie cardiaque, pas d'USC

In [17]:
sc %>% tbl('i2b2_observation_ufr') %>% 
    filter(location_cd %in% liste_reas) %>%
    count() %>% as_tibble()

n
<dbl>
14411


### Séjours de réanimation avec un codage FA dans la visite

In [18]:
sc %>% tbl('i2b2_observation_ufr') %>% 
    filter(location_cd %in% liste_reas) %>%
    semi_join(codages_fa, by = c('encounter_num', 'patient_num')) %>%
    count() %>% as_tibble()

n
<dbl>
12150


### Séjours de réanimation avec un codage FA dans la visite et des examens de laboratoire

In [19]:
sc %>% tbl('i2b2_observation_ufr') %>% 
    filter(location_cd %in% liste_reas) %>%
    semi_join(codages_fa, by = c('encounter_num', 'patient_num')) %>%
    semi_join(sc %>% tbl('i2b2_observation_lab'), by = c('encounter_num', 'patient_num')) %>%
    count() %>% as_tibble()

n
<dbl>
9417


### Séjours de réanimation avec un codage FA dans la visite, des examens de laboratoire, une durée comprise entre 2 jours et un an

In [20]:
sc %>% tbl('i2b2_observation_ufr') %>% 
    filter(location_cd %in% liste_reas) %>%
    semi_join(codages_fa, by = c('encounter_num', 'patient_num')) %>%
    semi_join(sc %>% tbl('i2b2_observation_lab'), by = c('encounter_num', 'patient_num')) %>%
    left_join(sc %>% tbl('i2b2_visit') %>% select(encounter_num, start_date, end_date), by = "encounter_num", suffix = c("_stay", "_visit")) %>%
    mutate(duree_sejour = ceiling((unix_timestamp(end_date_stay) - unix_timestamp(start_date_stay))/86400)) %>%
    filter(duree_sejour >= 2 && duree_sejour < 365) %>%
    count() %>% as_tibble()

n
<dbl>
7395


### Séjours uniques de réanimation avec un codage FA dans la visite, des examens de laboratoire, une durée comprise entre 2 jours et un an

In [21]:
sc %>% tbl('i2b2_observation_ufr') %>% 
    filter(location_cd %in% liste_reas) %>%
    semi_join(codages_fa, by = c('encounter_num', 'patient_num')) %>%
    semi_join(sc %>% tbl('i2b2_observation_lab'), by = c('encounter_num', 'patient_num')) %>%
    left_join(sc %>% tbl('i2b2_visit') %>% select(encounter_num, start_date, end_date), by = "encounter_num", suffix = c("_stay", "_visit")) %>%
    mutate(duree_sejour = ceiling((unix_timestamp(end_date_stay) - unix_timestamp(start_date_stay))/86400)) %>%
    filter(duree_sejour >= 2 && duree_sejour < 365) %>%
    group_by(patient_num) %>%
    filter(start_date_stay == min(start_date_stay)) %>%
    ungroup() %>%
    count() %>% as_tibble() 

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


n
<dbl>
6062


### Séjours uniques de réanimation avec un codage FA dans la visite, des examens de laboratoire, une durée comprise entre 2 jours et un an et après avoir exclu les diagnostics interférant avec la décision d'anticoagulation

In [22]:
sejours_rea_48_heures %>%
    count() %>% as_tibble()

n
<dbl>
4962
